Date: Oct 6, 1010

Author: Yue 'Alex' Fu

Group Member: Saniya, Florence, Varun, Alex

Project: DSCI 511

### Read Company names in S&P 500

In [2]:
import pandas as pd
SPdata = pd.read_csv('S&P 500 companies.csv',index_col=False).sort_values(by = 'industry')
SPdata

,ticker,company,industry
252,IPG,Interpublic Group,Advertising
353,OMC,Omnicom Group,Advertising
209,GD,General Dynamics,Aerospace & Defense
292,LMT,Lockheed Martin Corp.,Aerospace & Defense
446,TDG,TransDigm Group,Aerospace & Defense
...,...,...,...
461,URI,"United Rentals, Inc.",Trading Companies & Distributors
352,ODFL,Old Dominion Freight Line,Trucking
262,JBHT,J. B. Hunt Transport Services,Trucking
33,AWK,American Water Works Company Inc,Water Utilities


In [3]:
# sort the list by the sector
SPdata.industry.value_counts()

Health Care Equipment            20
Industrial Machinery             14
Semiconductors                   13
Electric Utilities               13
Packaged Foods & Meats           12
                                 ..
Health Care Technology            1
Hotel & Resort REITs              1
Computer & Electronics Retail     1
Gold                              1
Agricultural & Farm Machinery     1
Name: industry, Length: 124, dtype: int64

In [4]:
# Save the ticker and company name into a dictionary
SPdata_namedict = {SPdata.ticker[i]:SPdata.company[i] for i in range(len(SPdata.ticker))}
SPdata_namedict

{'MMM': '3M Company',
 'ABT': 'Abbott Laboratories',
 'ABBV': 'AbbVie Inc.',
 'ABMD': 'ABIOMED Inc',
 'ACN': 'Accenture plc',
 'ATVI': 'Activision Blizzard',
 'ADBE': 'Adobe Inc.',
 'AMD': 'Advanced Micro Devices Inc',
 'AAP': 'Advance Auto Parts',
 'AES': 'AES Corp',
 'AFL': 'AFLAC Inc',
 'A': 'Agilent Technologies Inc',
 'APD': 'Air Products & Chemicals Inc',
 'AKAM': 'Akamai Technologies Inc',
 'ALK': 'Alaska Air Group Inc',
 'ALB': 'Albemarle Corp',
 'ARE': 'Alexandria Real Estate Equities',
 'ALXN': 'Alexion Pharmaceuticals',
 'ALGN': 'Align Technology',
 'ALLE': 'Allegion',
 'LNT': 'Alliant Energy Corp',
 'ALL': 'Allstate Corp',
 'GOOGL': 'Alphabet Inc. (Class A)',
 'GOOG': 'Alphabet Inc. (Class C)',
 'MO': 'Altria Group Inc',
 'AMZN': 'Amazon.com Inc.',
 'AMCR': 'Amcor plc',
 'AEE': 'Ameren Corp',
 'AAL': 'American Airlines Group',
 'AEP': 'American Electric Power',
 'AXP': 'American Express Co',
 'AIG': 'American International Group',
 'AMT': 'American Tower Corp.',
 'AWK': 'Am

### Check the policy and robot.txt

In [5]:
import urllib.request

url = "http://finance.yahoo.com/robots.txt"

# Make the request
req = urllib.request.Request(url = url)

# Open the URL
handler = urllib.request.urlopen(req)

# Read/view the data as a string
robots = handler.read().decode('utf-8')
print(robots)

User-agent: *
Sitemap: https://finance.yahoo.com/sitemap_en-us_desktop_index.xml
Sitemap: https://finance.yahoo.com/sitemaps/finance-sitemap_index_US_en-US.xml.gz
Sitemap: https://finance.yahoo.com/sitemaps/finance-sitemap_googlenewsindex_US_en-US.xml.gz
Disallow: /m/
Disallow: /r/
Disallow: /__rapidworker-1.2.js
Disallow: /__blank
Disallow: /_td_api
Disallow: /_remote



### Prepare functions

In [6]:
!pip install selenium
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup

In [7]:
def start_chrome():
    # Start chrome
    driver = webdriver.Chrome(executable_path = r"C:\Users\even4\Anaconda3\Lib\site-packages\chromedriver_binary\chromedriver.exe")
    
    # Scrape from Yahoo Finance.
    address = "https://finance.yahoo.com/"
    
    # open Yahoo Finance
    driver.get(address)
    time.sleep(2)
    return driver

### Get sector for all 500 companies


In [9]:
def get_companyInfoContent(driver):
    # open the historical data
    driver.find_element_by_xpath("//span[text() = 'Profile']").click()
    time.sleep(2)

    # copy the content of the webpage and print
    content = driver.page_source

    HTMLcontent = BeautifulSoup(content, 'html.parser')

    # find the company information
    data = HTMLcontent.find('div', class_='Mb(25px)')

    record1 = data.find_all('p',class_='D(ib) W(47.727%) Pend(40px)')
    record2 = data.find_all('p',class_='D(ib) Va(t)')
    
    return record1, record2

In [10]:
def get_companyInfo(driver, company):
    record1, record2 = get_companyInfoContent(driver)
    df = []
    info = {}

    # sector 1
    txt = record1[0].get_text(strip=True, separator='|').split('|')
    info['Country'] = txt[-3]
    info['Contact'] = txt[-2]
    info['Website'] = txt[-1]
    info['Address_Line1'] = txt[0]
    
    if info['Country'] == 'United States':
        info['City'] = txt[-4].split(',')[0]
        info['State'] = txt[-4].split(',')[1].strip().split()[0]
        info['Zipcode'] = txt[-4].split(',')[1].strip().split()[1]
        if len(txt) == 5:
            info['Address_Line2'] = ' '
        elif len(txt) == 6:
            info['Address_Line2'] = txt[1]
    else:
        lineNum = len(txt)
        info['Address_Line2'] = ''
        for i in range(len(txt)-4-1):
            info['Address_Line2'] = info['Address_Line2'] + txt[i+1]
    
    # sector 2
    data = record2[0].find_all('span')
    info['Sector'] = data[1].text
    info['Industry'] = data[3].text
    info['Full_Time_Employees'] = data[5].text
    df.append(info)
    df = pd.DataFrame(df)
    return df

In [11]:

company_list = list(SPdata.ticker)

driver = start_chrome()

# extract company basic information
allInfo = pd.DataFrame(columns = ['Ticker','Company','Address_Line1','Address_Line2','City','Zipcode','Country','Contact','Website','Sector','Industry','Full_Time_Employees'])
exception = []
for company in company_list:
    try:
        # open target company
        driver.find_element_by_xpath("//input[@placeholder = 'Search for news, symbols or companies']").send_keys(company)
        time.sleep(2)
        driver.find_element_by_xpath("//button[@id= 'header-desktop-search-button']").click()
        time.sleep(2)
        
        info = get_companyInfo(driver, company)
        info['Ticker'] = company
        info['Company'] = SPdata_namedict[company]
        allInfo = pd.concat([allInfo, info])
    except:
        try:
            ticker = company
            company = SPdata_namedict[company]
            
            # open target company
            driver.find_element_by_xpath("//input[@placeholder = 'Search for news, symbols or companies']").send_keys(company)
            time.sleep(2)
            driver.find_element_by_xpath("//button[@id= 'header-desktop-search-button']").click()
            time.sleep(2)
            
            # Find company information
            info = get_companyInfo(driver, company)
            info['Ticker'] = ticker
            info['Company'] = company
            allInfo = pd.concat([allInfo, info])
        except Exception as e:
            exception.append(company)
            
allInfo.to_csv('SP500_allInfo.csv', index = False)

In [42]:
allInfo.Country.value_counts()

United States     497
Ireland             4
United Kingdom      2
Bermuda             1
Name: Country, dtype: int64

In [40]:
allInfo[allInfo.Country!='United States']

,Address_Line1,Address_Line2,City,Company,Contact,Country,Full_Time_Employees,Industry,Sector,State,Ticker,Website,Zipcode
0,5 Hanover Quay,Grand Canal Dock,NaN,Aptiv PLC,353 1 259 7013,Ireland,"141,000",Auto Parts,Consumer Cyclical,NaN,APTV,http://www.aptiv.com,NaN
0,Iveagh Court,Block D Harcourt Road,NaN,Allegion,353 1 254 6200,Ireland,"11,000",Security & Protection Services,Industrials,NaN,ALLE,http://www.allegion.com,NaN
0,Eaton House,30 Pembroke Road,NaN,Eaton Corporation,353 1 637 2900,Ireland,"93,000",Specialty Industrial Machinery,Industrials,NaN,ETN,http://www.eaton.com,NaN
0,The Priestley Centre,10 Priestley Road Surrey Research Park,NaN,Linde plc,44 14 8324 2200,United Kingdom,"76,662",Specialty Chemicals,Basic Materials,NaN,LIN,http://www.linde.com,NaN
0,The Leadenhall Building,The Aon Centre 122 Leadenhall Street,NaN,Aon plc,44 20 7623 5500,United Kingdom,"50,000",Insurance Brokers,Financial Services,NaN,AON,http://www.aon.com,NaN
0,The Sharp Building,Hogan Place,NaN,Perrigo,353 1 709 4000,Ireland,"11,200",Drug Manufacturers—Specialty & Generic,Healthcare,NaN,PRGO,http://www.perrigo.com,NaN
0,Seon Place,4th Floor 141 Front Street PO Box HM 845,NaN,Everest Re Group Ltd.,441-295-0006,Bermuda,"1,603",Insurance—Reinsurance,Financial Services,NaN,RE,http://www.everestre.com,NaN


### Get target company list

In [12]:
# sort the list by the sector
import pandas as pd
allInfo = pd.read_csv('SP500_allInfo.csv',index_col = False)
allInfo.sort_values(by = 'Sector')

,Address_Line1,Address_Line2,City,Company,Contact,Country,Full_Time_Employees,Industry,Sector,State,Ticker,Website,Zipcode
477,1915 Rexford Road,,Charlotte,Nucor Corp.,704 366 7000,United States,"26,800",Steel,Basic Materials,NC,NUE,http://www.nucor.com,28211
467,974 Centre Road,Building 730,Wilmington,DuPont de Nemours Inc,302 774 1000,United States,"35,000",Chemicals,Basic Materials,DE,DD,http://www.dupont.com,19805
465,4250 Congress Street,Suite 900,Charlotte,Albemarle Corp,980-299-5700,United States,"6,000",Specialty Chemicals,Basic Materials,NC,ALB,http://www.albemarle.com,28209
124,200 South Wilcox Drive,,Kingsport,Eastman Chemical,423 229 2000,United States,"14,500",Chemicals,Basic Materials,TN,EMN,http://www.eastman.com,37662
468,1221 McKinney Street,Suite 300,Houston,LyondellBasell,713 309 7200,United States,"19,100",Specialty Chemicals,Basic Materials,TX,LYB,http://www.lyondellbasell.com,77010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,1 Riverside Plaza,,Columbus,American Electric Power,614-716-1000,United States,"17,408",Utilities—Regulated Electric,Utilities,OH,AEP,http://www.aep.com,43215-2373
139,231 West Michigan Street,,Milwaukee,WEC Energy Group,414-221-2345,United States,"7,500",Utilities—Regulated Electric,Utilities,WI,WEC,http://www.wecenergygroup.com,53203
244,4300 Wilson Boulevard,11th Floor,Arlington,AES Corp,703-522-1315,United States,"8,000",Utilities—Diversified,Utilities,VA,AES,http://www.aes.com,22203
129,"30 Ivan Allen Jr. Boulevard, N.W.",,Atlanta,Southern Company,404-506-5000,United States,"27,943",Utilities—Regulated Electric,Utilities,GA,SO,http://www.southerncompany.com,30308


In [13]:
# check the number of companies in each sector
allInfo.Sector.value_counts()

Consumer Cyclical         221
Industrials                53
Technology                 45
Financial Services         39
Healthcare                 37
Real Estate                24
Consumer Defensive         21
Communication Services     17
Utilities                  17
Basic Materials            15
Energy                     15
Name: Sector, dtype: int64

In [14]:
# extract target sector
finalList = allInfo[allInfo.Sector == 'Technology']
finalList

,Address_Line1,Address_Line2,City,Company,Contact,Country,Full_Time_Employees,Industry,Sector,State,Ticker,Website,Zipcode
8,1049 Camino Dos Rios,,Thousand Oaks,Teledyne Technologies,805 373 4545,United States,"11,790",Scientific & Technical Instruments,Technology,CA,TDY,http://www.teledyne.com,91360-2362
39,Salesforce Tower,3rd Floor 415 Mission Street,San Francisco,Salesforce.com,415 901 7000,United States,"54,255",Software—Application,Technology,CA,CRM,http://www.salesforce.com,94105
41,690 East Middlefield Road,,Mountain View,Synopsys Inc.,650 584 5000,United States,"13,896",Software—Infrastructure,Technology,CA,SNPS,http://www.synopsys.com,94043
43,60 East Rio Salado Parkway,Suite 1000,Tempe,NortonLifeLock,650-527-8000,United States,"3,600",Software—Infrastructure,Technology,AZ,NLOK,http://www.nortonLifeLock.com,85281
44,111 McInnis Parkway,,San Rafael,Autodesk Inc.,415 507 5000,United States,"10,100",Software—Application,Technology,CA,ADSK,http://www.autodesk.com,94903
45,2655 Seely Avenue,Building 5,San Jose,Cadence Design Systems,408 943 1234,United States,"8,100",Software—Application,Technology,CA,CDNS,http://www.cadence.com,95134
48,851 West Cypress Creek Road,,Fort Lauderdale,Citrix Systems,954 267 3000,United States,"8,400",Software—Application,Technology,FL,CTXS,http://www.citrix.com,33309
85,1133 Innovation Way,,Sunnyvale,Juniper Networks,408 745 2000,United States,"9,838",Communication Equipment,Technology,CA,JNPR,http://www.juniper.net,94089
86,801 5th Avenue,,Seattle,F5 Networks,206-272-5555,United States,"5,786",Software—Infrastructure,Technology,WA,FFIV,http://www.f5.com,98104
87,500 West Monroe Street,Suite 4400,Chicago,Motorola Solutions Inc.,847 576 5000,United States,"17,000",Communication Equipment,Technology,IL,MSI,http://www.motorolasolutions.com,60661


In [15]:
finalList.to_csv('FinalList_Tech.csv', index = False)

### Scrape historical data

In [16]:
def get_historicalContent(driver, company):

    # open the historical data
    driver.find_element_by_xpath("//span[text() = 'Historical Data']").click()
    time.sleep(2)

    # open the time period button
    driver.find_element_by_xpath('//span[@class="C($linkColor) Fz(14px)"]').click()
    time.sleep(2)

    # click 5 years
    driver.find_element_by_xpath('//button[@data-value="5_Y"]').click()
    time.sleep(2)

    # click apply
    driver.find_element_by_xpath('//span[text() = "Apply"]').click()
    time.sleep(2)


    # scroll down three times to load the 5 year table.
    for i in range(15):
        driver.execute_script("window.scrollBy(0,5000)")
        time.sleep(2)

    # copy the content of the webpage and print
    content = driver.page_source

    HTMLcontent = BeautifulSoup(content, 'html.parser')

    # Find the table of historical data
    data = HTMLcontent.find('table', class_='W(100%) M(0)')

    # find all rows in the table
    records = data.find_all('tr', class_='BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)')
    return driver, records

In [17]:
def get_historicalData(driver, company):
    driver, records = get_historicalContent(driver, company)
    df = []
    for i in range(0, len(records)):
        try:
            row = {}
            price = records[i].find_all('td')

            if len(price) == 7:
                row["Date"] = price[0].find('span').text.replace(',', '')
                row["Open"] = price[1].find('span').text.replace(',', '')
                row["High"] = price[2].find('span').text.replace(',', '')
                row["Low"] = price[3].find('span').text.replace(',', '')
                row["Close"] = price[4].find('span').text.replace(',', '')
                row["Adj Close"] = price[5].find('span').text.replace(',', '')
                row["Volume"] = price[6].find('span').text.replace(',', '')
                df.append(row)
        except:
            print("Row Number: " + str(i))
        i = i + 1

        # Converted list of dictionaries to a Dataframe.
    df = pd.DataFrame(df)

    return df

In [54]:
# try one company first
company_list = list(finalList.Ticker)

driver = start_chrome()


# extract the historical data
allData = pd.DataFrame(columns = ['Ticker','Company','Date','Open','High','Low','Close','Adj Close','Volume'])
exception_data = []
for company in company_list:
    try:
        print(company)
        
        # open target company
        driver.find_element_by_xpath("//input[@placeholder = 'Search for news, symbols or companies']").send_keys(company)
        time.sleep(2)
        driver.find_element_by_xpath("//button[@id= 'header-desktop-search-button']").click()
        time.sleep(2)
        
        data = get_historicalData(driver, company)
        data['Ticker'] = [company] * len(data.Date)
        data['Company'] = [SPdata_namedict[company]] * len(data.Date)
        allData = pd.concat([allData,data])
    except:
        try:
            ticker = company
            company = SPdata_namedict[company]
            print(company)
            driver = start_chrome()
            # open target company
            driver.find_element_by_xpath("//input[@placeholder = 'Search for news, symbols or companies']").send_keys(company)
            time.sleep(2)
            driver.find_element_by_xpath("//button[@id= 'header-desktop-search-button']").click()
            time.sleep(2)
            
            data = get_historicalData(driver, company)
            data['Ticker'] = [company] * len(data.Date)
            data['Company'] = [SPdata_namedict[company]] * len(data.Date)
            allData = pd.concat([allData,data])
        except Exception as e:
            exception_data.append(company)

In [55]:
allData.to_csv('StockPrice_Tech_FiveYear.csv',index = False)

45

In [57]:
allData.shape

Adj Close    0
Close        0
Company      0
Date         0
High         0
Low          0
Open         0
Ticker       0
Volume       0
dtype: int64

In [58]:
allData.Ticker.value_counts()

INTC    1259
LRCX    1259
JNPR    1259
XRX     1259
NOW     1259
CDNS    1259
MSI     1259
AKAM    1259
SNPS    1259
FIS     1259
XLNX    1259
ADI     1259
TDY     1259
MU      1259
NVDA    1259
MSFT    1259
CTXS    1259
FTNT    1259
AVGO    1259
CTSH    1259
KLAC    1259
CRM     1259
NTAP    1259
AAPL    1259
CSCO    1259
IPGP    1259
FLT     1259
ADSK    1259
MCHP    1259
KEYS    1259
MXIM    1259
JKHY    1259
FFIV    1259
IT      1259
TXN     1259
HPQ     1259
ZBRA    1259
AMAT    1259
FISV    1259
QCOM    1259
SWKS    1259
NLOK    1258
FTV     1110
BR       140
VNT       35
Name: Ticker, dtype: int64